In [2]:
import os
import json
import hashlib
import uuid

# Đường dẫn đến thư mục gốc
root_dir = "D:\\crawl-data\\tiki"

# Đường dẫn đến thư mục lưu trữ file JSON
output_dir = "data"

# Tạo thư mục lưu trữ file JSON nếu chưa tồn tại
os.makedirs(output_dir, exist_ok=True)

In [8]:
# Tạo các từ điển để lưu trữ và quản lý dữ liệu
user_data = []
book_data = []
review_data = []
category_data = []

# Từ điển lưu trữ thông tin người dùng theo tên
user_dict = {}

# Từ điển theo dõi đánh giá của từng tên người dùng
user_reviews_tracker = {}


# Hàm so sánh người dùng
def is_same_user(existing_user, new_user):
    """
    Kiểm tra xem hai người dùng có phải là cùng một người không
    Điều kiện:
    - Nếu ngày tham gia giống nhau thì số lượng đánh giá phải tăng
    - Nếu ngày tham gia mới lớn hơn thì số lượng đánh giá phải tăng
    - Nếu ngày tham gia mới nhỏ hơn thì số lượng đánh giá phải giảm
    """
    existing_date = existing_user.get("date", "")
    new_date = new_user.get("date", "")

    existing_review_count = existing_user.get("review_count", 0)
    new_review_count = new_user.get("review_count", 0)

    # Nếu ngày tham gia giống nhau
    if existing_date == new_date:
        return new_review_count > existing_review_count

    # Nếu ngày tham gia mới lớn hơn
    if new_date > existing_date:
        return new_review_count > existing_review_count

    # Nếu ngày tham gia mới nhỏ hơn
    if new_date < existing_date:
        return new_review_count < existing_review_count

    return False


# Duyệt qua tất cả các thư mục con trong thư mục gốc
for dir_name in os.listdir(root_dir):
    category_id = str(uuid.uuid4())
    category_data.append(
        {
            "categoryId": category_id,
            "name": dir_name,
            "slug": dir_name,
        }
    )

    dir_path = os.path.join(root_dir, dir_name)
    if os.path.isdir(dir_path):
        # Duyệt qua tất cả các tệp tin trong thư mục con
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            if file_name.endswith(".json"):
                # Đọc dữ liệu từ tệp tin JSON
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                book_id = str(uuid.uuid4())
                book_data.append(
                    {
                        "bookId": book_id,
                        "categoryId": category_id,
                        "info": data["info"],
                        "details": data["details"],
                        "description": data["description"],
                    }
                )

                # Xử lý đánh giá
                for idx, review in enumerate(data["reviews"]):
                    reviewer = review["reviewer"]
                    reviewData = review["review"]
                    name = reviewer["name"]

                    # Khởi tạo theo dõi cho tên người dùng nếu chưa tồn tại
                    if name not in user_reviews_tracker:
                        user_reviews_tracker[name] = []

                    # Chuẩn bị thông tin người dùng mới
                    new_user_info = {
                        "name": name,
                        "date": reviewer.get("date", ""),
                        "review_count": len(user_reviews_tracker[name]) + 1,
                    }

                    # Kiểm tra xem người dùng đã tồn tại chưa
                    is_existing_user = False
                    existing_user_index = -1

                    # Chỉ lấy đánh giá đầu tiên của mỗi tên người dùng cho mỗi sách
                    book_reviews_by_name = [
                        r for r in user_reviews_tracker[name] if r["book_id"] == book_id
                    ]
                    if book_reviews_by_name:
                        continue

                    # Tìm người dùng tồn tại với cùng thông tin
                    for i, existing_user in enumerate(user_reviews_tracker[name]):
                        if is_same_user(existing_user, new_user_info):
                            is_existing_user = True
                            existing_user_index = i
                            break

                    # Xử lý logic thêm người dùng và review
                    if not is_existing_user:
                        # Tạo người dùng mới
                        user_id = str(uuid.uuid4())
                        user_dict[f"{name}_{len(user_reviews_tracker[name])}"] = {
                            "userId": user_id,
                            "name": name,
                            "email": f"{user_id.replace('-', '_')}@gmail.com",
                            "password": f"{user_id.replace('-', '_')}@gmail.com",
                            "date": reviewer.get("date", ""),
                        }

                        # Thêm review
                        review_data.append(
                            {
                                "bookId": book_id,
                                "userId": user_id,
                                "rating": reviewData["rating"],
                                "comment": reviewData["content"],
                            }
                        )

                        # Cập nhật theo dõi đánh giá
                        user_reviews_tracker[name].append(
                            {
                                "book_id": book_id,
                                "date": reviewer.get("date", ""),
                                "review_count": len(user_reviews_tracker[name]) + 1,
                            }
                        )

# Ghi dữ liệu ra các file JSON
user_data = list(user_dict.values())

# Ghi các file JSON như bình thường
user_output_file = os.path.join(output_dir, "user_data.json")
with open(user_output_file, "w", encoding="utf-8") as jsonfile:
    json.dump(user_data, jsonfile, ensure_ascii=False, indent=4)

category_output_file = os.path.join(output_dir, "category_data.json")
with open(category_output_file, "w", encoding="utf-8") as jsonfile:
    json.dump(category_data, jsonfile, ensure_ascii=False, indent=4)

book_output_file = os.path.join(output_dir, "book_data.json")
with open(book_output_file, "w", encoding="utf-8") as jsonfile:
    json.dump(book_data, jsonfile, ensure_ascii=False, indent=4)

review_output_file = os.path.join(output_dir, "review_data.json")
with open(review_output_file, "w", encoding="utf-8") as jsonfile:
    json.dump(review_data, jsonfile, ensure_ascii=False, indent=4)

In [11]:
import os
import json
import csv

# Thư mục chứa các tệp JSON đầu vào
input_dir = "D:\\crawl-data\\data"
# Thư mục chứa các tệp CSV đầu ra
output_dir = "D:\\crawl-data\\data"


# Hàm để ghi dữ liệu JSON vào CSV
def write_json_to_csv(data, output_file, fieldnames):
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(
            csvfile, fieldnames=fieldnames
        )
        writer.writeheader()
        writer.writerows(data)


# Đọc các tệp JSON đầu vào
user_file = os.path.join(input_dir, "user_data.json")
category_file = os.path.join(input_dir, "category_data.json")
book_file = os.path.join(input_dir, "book_data.json")
review_file = os.path.join(input_dir, "review_data.json")

# Tải và chuyển đổi dữ liệu người dùng
with open(user_file, "r", encoding="utf-8") as f:
    user_data = json.load(f)
    user_output_file = os.path.join(output_dir, "user_data.csv")
    user_fieldnames = ["userId", "name", "email", "password", "date"]
    write_json_to_csv(user_data, user_output_file, user_fieldnames)

# Tải và chuyển đổi dữ liệu danh mục
with open(category_file, "r", encoding="utf-8") as f:
    category_data = json.load(f)
    category_output_file = os.path.join(output_dir, "category_data.csv")
    category_fieldnames = ["categoryId", "name", "slug"]
    write_json_to_csv(category_data, category_output_file, category_fieldnames)

# Tải và chuyển đổi dữ liệu sách
with open(book_file, "r", encoding="utf-8") as f:
    book_data = json.load(f)
    book_output_file = os.path.join(output_dir, "book_data.csv")
    book_fieldnames = [
        "bookId",
        "categoryId",
        "info",
        "details",
        "description",
    ]
    write_json_to_csv(book_data, book_output_file, book_fieldnames)

# Tải và chuyển đổi dữ liệu đánh giá
with open(review_file, "r", encoding="utf-8") as f:
    review_data = json.load(f)
    review_output_file = os.path.join(output_dir, "review_data.csv")
    review_fieldnames = ["bookId", "userId", "rating", "comment"]
    write_json_to_csv(review_data, review_output_file, review_fieldnames)

print("Chuyển đổi JSON sang CSV hoàn tất.")

Chuyển đổi JSON sang CSV hoàn tất.
